# Plot ERE obtained by Lüscher formalism from database 

## Init

In [ ]:
import os
from itertools import product

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt

from luescher_nd.zeta.zeta3d import Zeta3D

from luescher_nd.database.connection import DatabaseSession
from luescher_nd.database.tables import LongRangeEnergyEntry
from luescher_nd.database.tables import ContactEnergyEntry

from luescher_nd.hamiltonians.kinetic import HBARC

In [ ]:
import matplotlib
import re

markers = re.findall('``"([a-z]+)"``', matplotlib.markers.__doc__)

In [ ]:
%load_ext blackcellmagic

In [ ]:
DATA = os.path.join(os.getcwd(), os.pardir, "data")

In [ ]:
def get_energy_frame(database: str, round_digits:int=2):
    """Extracts data frame from table
    """
    db = os.path.abspath(os.path.join(DATA, database))
    sess = DatabaseSession(db, commit=False)
    df = pd.read_sql_table("energy", sess.engine, index_col="id").drop(
        columns=["comment"]
    )

    df["L"] = df["n1d"] * df["epsilon"]
    df["nstep"] = df["nstep"].fillna(-1).astype(int)
    df["E [MeV]"] = df["E"] * HBARC
    df["p2"] = df["E"] * 2 * df["mass"] / 2
    df["x"] = df["p2"] / (2 * np.pi / df["L"]) ** 2

    integer_x = [x for x in range(int(df.x.min())-1, int(df.x.max()+2)) if x !=0]
    df = df[~df.x.round(round_digits).isin(integer_x)]

    for L, n1d, nstep in list(
        product(df.L.unique(), df.n1d.unique(), df.nstep.unique())
    ):
        epsilon = L / n1d
        ind = (df.L == L) & (df.n1d == n1d) & (df.nstep == nstep)
        z = Zeta3D(L, epsilon, nstep if nstep > 0 else None)
        df.loc[ind, "y"] = np.abs(z(df.loc[ind, "x"].values)) / np.pi / df.loc[ind, "L"]

    df = df.dropna()

    df.loc[df.nstep == -1, "nstep"] = "$\infty$"
    df["nstep"] = df["nstep"].astype(str)

    return df


## Plotting

Load the data frame

In [ ]:
DB_NAME = "db-contact-fv-d-fitted.sqlite"

df = get_energy_frame(DB_NAME)
df.head()

Specify grid properties

In [ ]:
row = "$L$ [fm$^{-1}$]"
hue = "epsilon"
col = "nstep"
x = "$p^2$ [fm$^{-2}$]"
y = "$p \cot(\delta_0(p))$ [fm$^{-1}$]"

Specify data ranges

In [ ]:
query = "x > -5"

and plot

In [ ]:
tf = df.query(query).sort_values("x")
for key, val in {
    "y": "$p \cot(\delta_0(p))$ [fm$^{-1}$]",
    "L": "$L$ [fm$^{-1}$]",
    "p2": "$p^2$ [fm$^{-2}$]",
}.items():
    tf[val] = tf[key]

grid = sns.FacetGrid(
    data=tf,
    col=col,
    hue=hue,
    row=row,
    sharex="row",
    sharey=True,
    legend_out=True,
    hue_kws={"marker": markers},
    palette="cubehelix",
)

grid.map(plt.plot, x, y, ms=3, lw=0.5, ls="--", marker="s")

for ax in grid.axes.flatten():
    ax.set_yscale("log")

grid.fig.set_dpi(200)
grid.add_legend(frameon=False)

plt.show()


In [ ]:
grid.savefig(
    "figs/" + DB_NAME.replace("db-", "").replace("sqlite", "pdf"), bbox_inches="tight"
)